In [2]:
# Load modules
import os
import pandas as pd
import re
import ast
import glob
import csv

import shutil
import numpy as np
import geopy
import pygmt
import pyproj
from geopy.distance import geodesic
from geopy.distance import distance
from os import system as sys

path = "C:/Users/19049/coding_files/data/CFM5.3_traces.ts"
subset = "C:/Users/19049/coding_files/data/subset.ts"
folder = "C:/Users/19049/coding_files/faults/"
sub_folder = "C:/Users/19049/coding_files/faults/wb"
wb_2b = "C:/Users/19049/coding_files/wb_2b/"

to_open = path

In [3]:
#we are iterating through each line or row of the CFM5.3traces.ts file
#there is a "> " at the beginning of each line within the data file that I'm using to identify that the current line is a fault name
#flags the type of line currently being evaluated
#List that tracks the 'yes' and 'no' answers of whether the fault name is one included in faults_to_include- lets us ignore the fault data we don't want but it needs to be reset after each fault real_name check
#(These lines check each provided name in faults_to_include to the fault name being evaluated
#progress tracker, since this takes a second on my machine
#We want to make a new df for each fault
#we only need one 'yes' to indicate that this is a fault we want
#there will be a number of 'no' in each fault tracker list equal to the number of faults_to_include elements for those we don't want, and -1 for those we do
#There are 15 #s at the beginning of the file that we want to explicitly avoid
#flags the type of line currently being evaluated
#The while loop is needed because with the is_number flag we can go through an ambiguous number of lines underneath each fault name until we reach a new line beginning with "> " where we will reset everything an begin a new df
#if there is one 'yes' answer within the tracker list then it will adjust to df: purpose of this is to only take data from a fault we want and ignore the rest
#adjusting to df translates the single string line to a list of floats and adjusts the easting and northing values by -345000 and -3870000 respectively
#Saves the current line of adjusted values to a csv under the current real_name of the fault. make sure is in line with while, that gets messed up a lot

In [4]:
#13 faults that are included in the project that we want to make worlbuilder files of
# format: '*_fault',\n
faults_to_include = [
    'Panamint_Valley_fault',
    'Southern_Death_Valley_fault',
    'Black_Mountain_fault',
    'Owl_Lake_fault',
    'Hunter_Mountain_fault',
    'Ash_Hill_fault',
    'Independence_Sierra_Nevada_fault', #Gotta add the _fault identification bit cause of
    'Independence_fault',               #these two lines
    'Little_Lake_fault',
    'Airport_Lake_fault',
    'Eastern_Little_Lake_north_north_east_cross_fault',
    'Owens_Valley_fault',
    'Northern_Death_Valley_fault',
    'Casmalia_fault',
    'thing'                             #if thing shows up in files, stuff is wack
    ]

In [5]:
#makes an empty pandas dataframe to fill the adjusted values with
#",'Elevation(m)'"
def make_empty_fault_df():
    global fault_dataframe
    fault_dataframe = pd.DataFrame([],columns = ['Easting(m)','Northing(m)'])

#the line being read is delimited and turned from a line of strings into a line of floats
#then it is apended to the fault_dataframe dataframe with the elements adjusted
#add ", float_list[2]" to end of fault_dataframe to log the elevation
#fault_dataframe.loc[len(fault_dataframe)]
def adjust_to_df():
        string_line = line.split()
        float_list = list(map(float, string_line))
        fault_dataframe.loc[len(fault_dataframe)] = float_list[0]-345000, float_list[1]-3870000
        # print(bruh)
        
def adjust_to_wb():
        
        string_line = line.split()
        float_list = list(map(float, string_line))
        coors = [float_list[0]-345000, float_list[1]-3870000]
        coordinates.append(coors)  
        return coordinates
    
        

#takes the finished fault_dataframe df and names it the current fault trace, making it a csv    
def save_to_file():
    fault_dataframe.to_csv(folder+use_name+".csv")
def save_to_wb():
    wb.append(feature_string)

In [6]:
def model_feature_string(model, name, dip_point, coordinates):
    global feature_string
    feature_string = '"model":"' + str(model) + '", "name":"' + str(name) + '", "dip point":"' + str(dip_point) + '", "coordinates":' + str(coordinates) + ',\n'
    return feature_string

In [ ]:
generated_faults = glob.glob(folder+"/*")
for file in generated_faults:
    print(file)
    with open(file) as f:
        for line in f:
            print(line)

In [7]:
#works
with open(to_open, encoding='utf-8') as f:
    for line in f:                                          
        if line.startswith("> "):                           
            is_number = False                               
            included_fault_tracker = [] 
            
            for real_name in faults_to_include:                            
                if real_name in line:                       
                    print(real_name)
                    global use_name
                    use_name = real_name                                        
                    make_empty_fault_df()                   
                    included_fault_tracker.append('yes')     
                else:                                       
                    included_fault_tracker.append('no')     
        elif line.startswith("#"):                          
            pass                                            
        else:                                               
            is_number = True                                
            while is_number == True:                        
                if 'yes' in included_fault_tracker:         
                    adjust_to_df()                      
                    
                else:                           
                    pass
                break
            save_to_file()                                  

Black_Mountain_fault
Northern_Death_Valley_fault
Southern_Death_Valley_fault
Casmalia_fault
Casmalia_fault
Owl_Lake_fault
Ash_Hill_fault
Airport_Lake_fault
Hunter_Mountain_fault
Little_Lake_fault
Eastern_Little_Lake_north_north_east_cross_fault
Owens_Valley_fault
Panamint_Valley_fault
Independence_fault
Independence_Sierra_Nevada_fault


In [39]:
for file in glob.glob("C:/Users/19049/coding_files/*"):
    if "wb_wip.txt" in file:
        os.remove(file)

In [68]:
#this project is trying to read the data file and directly write to wb, but im scrapping it for now and moving to using the already generated files
#clears working file so we don't stack
for file in glob.glob("C:/Users/19049/coding_files/*"):
    if "wb_wip.txt" in file:
        os.remove(file)

wb=[]
model = '"model":"fault"'
version = '"version":"0.5"'
dip_point = '"dip point":[0,0]'
Associated_with_current_fault = True
coordinates = []
included_fault_tracker = []
global name

#determine last line
with open("C:/Users/19049/coding_files/data/subset.ts") as f:
    for line in f:
        pass
    global last_line
    last_line = line

#initialize the wb file
with open("wb_wip.txt", "a") as f:
    f.write('{')
    f.write('\n')
    f.write('\t')
    f.write(version)
    f.write(', \n')
    f.write('\t')
    f.write('"features":')
    f.write('\n')
    f.write('\t')
    f.write('[')
    f.write('\n')
    
with open("C:/Users/19049/coding_files/data/subset.ts") as f:
    for line in f:
        #purpose of this is that the last set of coordinates were not being outputted, since the output was dependent on a new fault being read in, so the above for loop finds the last line and this if statement outputs the stuff if it is the last line. Might be a stupid way of doing this but idk how else to do it.
        if line == last_line:
            Associated_with_current_fault = False
            is_number = False
            while Associated_with_current_fault == False:
                if 'yes' in included_fault_tracker:
                    with open("wb_wip.txt", "a") as f:
                        codename = '"name":'+name
                        f.write('\t')
                        f.write('\t')
                        f.write('{')
                        f.write(model)
                        f.write(',')
                        f.write(codename)
                        f.write(',')
                        f.write(dip_point)
                        f.write(', \n')
                        f.write('\t')
                        f.write('\t')
                        f.write('"coordinates":[')
                        f.write('\n')
                        f.write(', \n'.join(str(i) for i in coordinates))
                        f.write('\n')
                        f.write('\t')
                        f.write('\t')
                        f.write('],')
                        f.write('\n')
                        f.write('\t')
                        f.write('\t')
                        f.write('"segments" n stuff go here')
                        f.write('\n')
                        f.write('\t')
                        f.write('\t')
                        f.write('}') #this one should be the last one and doesn't have a ,
                        f.write('\n')
                        # f.write('\n')
                    print(coordinates)
                break
        #the beginning of each new fault will have this as the starting characters of that line and every coordinate until the start of the next fault will be associated with it, 
        if line.startswith("> "):
            Associated_with_current_fault = False
            is_number = False
            while Associated_with_current_fault == False:
                if 'yes' in included_fault_tracker:
                    with open("wb_wip.txt", "a") as f:
                        codename = '"name":'+name
                        f.write('\t')
                        f.write('\t')
                        f.write('{')
                        f.write(model)
                        f.write(',')
                        f.write(codename)
                        f.write(',')
                        f.write(dip_point)
                        f.write(', \n')
                        f.write('\t')
                        f.write('\t')
                        f.write('"coordinates":[')
                        f.write('\n')
                        f.write(', \n'.join(str(i) for i in coordinates))
                        f.write('\n')
                        f.write('\t')
                        f.write('\t')
                        f.write('],')
                        f.write('\n')
                        f.write('\t')
                        f.write('\t')
                        f.write('"segments" n stuff go here')
                        f.write('\n')
                        f.write('\t')
                        f.write('\t')
                        f.write('},') #has ,
                        f.write('\n')
                        f.write('\n')
                    print(coordinates)
                break
            included_fault_tracker = []

            for real_name in faults_to_include:                            
                if real_name in line:           
                    
                    name = real_name
                    coordinates = []
                    print(name)
                    included_fault_tracker.append('yes') 
                else:
                    included_fault_tracker.append('no')
            print(included_fault_tracker)
            
        elif line.startswith("#"):                          
            pass  
        
        else:                                               
            is_number = True 
            
            while is_number == True:                        
                if 'yes' in included_fault_tracker: 
                    #print('sure')
                    adjust_to_wb()
                else:                           
                    pass
                
                break
        Associated_with_current_fault = True  
        
with open("wb_wip.txt", "a") as f:
    f.write('\t')
    f.write(']')
    f.write('\n')
    f.write('}')


Black_Mountain_fault
['no', 'no', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no']
[[165833.0625, 173051.0], [165133.0, 172049.5], [164825.71879999997, 170926.5], [165760.71879999997, 169776.0], [166028.09379999997, 169092.0], [166643.6875, 168449.75], [167705.6875, 167060.5], [168115.6875, 165789.0], [168525.65620000003, 164517.5], [169018.53120000003, 163293.0], [169155.75, 162357.0], [169295.1875, 160998.0], [169989.21879999997, 159175.0], [170284.8125, 158736.25], [170526.375, 157920.25], [171062.25, 156609.0], [171771.25, 155463.5], [172487.71870000003, 154656.75], [173285.375, 153466.25], [173853.5, 151572.25], [174247.625, 151004.75], [174530.0, 149981.75], [175130.40629999997, 149006.5], [175110.375, 148103.0], [175598.46879999997, 147188.5], [176010.09370000003, 145372.0], [175980.03120000003, 144017.0], [175617.53120000003, 142951.5], [175822.0625, 141987.0], [176204.75, 141413.75], [176567.375, 139937.25], [176608.8125, 139258.5], [176469.593799

In [25]:
print(last_line)

156687.5625   3877500.7500   -76.058300000


In [ ]:
def model_feature_string(model_name, feature_name, min_depth, max_depth, coordinates, is_subducting, dip_point):
    
    '''
    Creates a string which is then written to a world builder file that initializes a world builder feature

    model_name    = string specifying the WB model type you want for the feature 
    feature_name  = user specified string naming the feature (can be anything, for ease of reading the WB file)
    min_depth     = minimum depth extent of the feature
    max_depth     = maximum depth extenet of the feautre
    coordinates   = array specifying the points which create a bounding volume of the feature
    is_subducting = True or False, whether the feature is a subducting plate
    dip_point     = if is_subducting=True, location of the dip point of the subducting slab    
    '''
    
    if is_subducting == False:
        feature_string = '"model":"' + str(model_name) + '", "name":"' + str(feature_name) + '", "min depth":' + str(min_depth) +', "max depth":' + \
                         str(max_depth) + ', "coordinates":' + str(coordinates) + ',\n'
    else:
        feature_string = '"model":"' + str(model_name) + '", "name":"' + str(feature_name) + '", "coordinates":' + str(coordinates) + ', "dip point":' + str(dip_point) + ',\n'
    return feature_string

In [ ]:
def segment_string(length, thickness, angle, top_truncation, coordinate, total_sections, segment_num, current_segment):

    '''
    Writes the string that defines segments of a slab to the world builder file

    When initializing a slab in Worldbuilder, you must specify the same number of segments for each section along strike
    of the slab, which is troublesome since some sections of slab are much longer than others. To get around this,
    the slab is initialized with the same number of segments as the section with the maximum number of segments. 
    Then, sections with less than this number of segments are assigned filler sections with lengths of 0 m until
    they reach the required number of segments

    length          = the length of a given segment
    thickness       = the thickness of a given segment
    angle           = the dip of a given segment
    coordinate      = the trench coordinate index coupled to the current section
    total_sections  = the number of sections making up the slab
    segment_num     = the number of segments making up each section
    current_segment = the current segment index of a section
    '''
 
    string_total = ''
    filler_segment_number = abs(len(thickness) - segment_num)
    if current_segment < filler_segment_number:
        if len(thickness[0]) > 1:
            # Here we proceed as above, adding the segment strings until we reach the end of the array
            string_total += '{"length":' + str(0.0) + ', "thickness":' + str([np.max(thickness), np.max(thickness)]) + ', "angle":' + str([0.0, 0.0]) + '},\n'# \
                           # ', "angle":' + str([0,0]) + '},\n'
            return string_total
        
        else:
            # Here we proceed as above, adding the segment strings until we reach the end of the array
            string_total += '{"length":' + str(0.0) + ', "thickness":' + str([np.max(thickness)]) + ', "angle":' + str([0.0, 0.0]) + '},\n'# \
                           # ', "angle":' + str([0,0]) + '},\n'
            return string_total
        
        # Now we have reached the end of the array, but need to fill out the amount of segments to reach the maximum
        # segment number. Add segments with lengths of 1m, thicknesses of 1m, and dips of 1 degree until this is the case.
    else:
        if current_segment != segment_num - 1:
            string_total = '{"length":' + str(length[current_segment - filler_segment_number]) + ', "thickness":' + str(thickness[current_segment - filler_segment_number]) + \
                            ', "angle":' + str(angle[current_segment - filler_segment_number]) + ', "top truncation":' + str(top_truncation) + '},\n'
            return string_total

        elif current_segment == (segment_num - 1) and coordinate == total_sections:
            string_total += '{"length":' + str(length[current_segment - filler_segment_number]) + ', "thickness":' + str(thickness[current_segment - filler_segment_number]) + \
                        ', "angle":' + str(angle[current_segment - filler_segment_number]) + ', "top truncation":' + str(top_truncation) + '}]}\n'
            return string_total

        else:
            string_total += '{"length":' + str(length[current_segment - filler_segment_number]) + ', "thickness":' + str(thickness[current_segment - filler_segment_number]) + \
                        ', "angle":' + str(angle[current_segment - filler_segment_number]) + ', "top truncation":' + str(top_truncation) + '}]},\n'
            return string_total

In [ ]:
def segment_section(world_builder_file, profile_directory, xshift, yshift, slab_thickness, top_truncation, coordinate_system):
    
    '''
    Calculates dip, thickness, and length of segments then uses segment_string() to create the string for the world builder file.
    This function has the option to vary the slab thickness between certain depths. If the varialbe slab_thickness is a scalar,
    a constant thickness is assumed for the entire section. To vary thickness, slab_thickness must be an array where each entry is an 
    array with length two. The entries of this array are the thickness of the slab and the depth to where 
    that thickness occurs. For example, to have the slab be 100km thick between 0 <= depth <= 200km, and then
    have the thickness increase to 150km for depth > 200km, the variable slab_thickness would need to be set as:

    slab_thickness = [ [100e3, 200e3], [150e3, 1e10] ]

    Where 1e10 was chosen to be a depth so high it would never be reached. Slab thickness and dip are varied gradually 
    down dip

    world_builder_file = the name of the world builder file
    profile_directory  = directory containing the world builder file
    xshift             = the amount of shift in the x direction, m
    yshift             = the amount of shift in the y direction, m
    slab_thickness     = the thickness of the slab, scaler for uniform thickness, array for variable, km
    '''
  
    # Import packages for spherical coordinate systems
    import geopy
    from geopy.distance import geodesic
    from geopy.distance import distance
    import pyproj
    geodesic = pyproj.Geod(ellps='WGS84')
    
    # Create an array which stores the length of each profile, the longest profile determines the number of
    # segments required for initializing the slab
    length = []
    for file in np.sort(os.listdir(profile_directory)):
        length.append(len(np.loadtxt(fname=profile_directory + file)))
    segment_num = max(length)
    total_sections = len(os.listdir(profile_directory)) - 1
    
    # This loop initializes the slab with the correct number of segments
    world_builder_file.write('"segments":[\n')
    for i in range(segment_num):
        if i != segment_num - 1:
            world_builder_file.write('{"length":0, "thickness":[0.0], "angle":[0]},\n')
        else:
            world_builder_file.write('{"length":0, "thickness":[0.0], "angle":[0]}],\n\n')

    # This loops through all sections and determines the thickness, dip, and length of each segment
    world_builder_file.write('    "sections":[')
    index = 0
    for file in np.sort(os.listdir(profile_directory)):
        index += 1
        filename = os.path.join(profile_directory, file)

        track_x = np.loadtxt(fname=filename, usecols=0) + xshift
        track_y = np.loadtxt(fname=filename, usecols=1) + yshift
        track_z = np.loadtxt(fname=filename, usecols=2)

        slab_length = 0
        segment_length = []
        dip_holder = [0]
        
        # Here we check to see if the thickness is set to vary along the slab by checking if slab_thickness is
        # a scalar or not. Dip is also computed and stored for output to the world builder file here
        if hasattr(slab_thickness, "__len__"):
            thick_holder = [slab_thickness[index - 1][:, 0][0]]
            for i in range(1, len(track_z)):
                thick_index = np.min(np.where( slab_thickness[index - 1][:, 1] > np.abs(track_z[i]) ))
                thick_holder.append(slab_thickness[index - 1][:, 0][thick_index])
                
                if coordinate_system == 'Cartesian':
                    slab_cart_proj = np.sqrt( (track_x[i] - track_x[i - 1])**2 + (track_y[i] - track_y[i - 1])**2 )
                    slab_length = np.sqrt( (track_x[i] - track_x[i - 1])**2 + (track_y[i] - track_y[i - 1])**2 + (track_z[i] - track_z[i - 1])**2)
                    riserun = (track_z[i - 1] - track_z[i]) / (slab_cart_proj)
                    
                elif coordinate_system == 'Spherical':
                    fwd_az, back_az, slab_cart_proj_m = geodesic.inv(track_x[i-1], track_y[i-1], track_x[i], track_y[i])
                    slab_cart_proj = slab_cart_proj_m / 1e3
                    slab_length = np.sqrt( slab_cart_proj**2 + (track_z[i] - track_z[i - 1])**2)
                    riserun = (track_z[i - 1] - track_z[i]) / (slab_cart_proj)
                    
                segment_length.append(slab_length * 1000)
                dip_holder.append(np.rad2deg(np.arctan(abs(riserun))))
                
            segment_thickness = []
            for k in range(len(thick_holder) - 1):
                if k != int(len(thick_holder) - 1):
                    segment_thickness.append([thick_holder[k], thick_holder[k + 1]])
                else:
                    segment_thickness.append([thick_holder[k]])
         
        else:
            segment_thickness = []
            for i in range(1, len(track_z)):
                segment_thickness.append([slab_thickness])
                
                if coordinate_system == 'Cartesian':
                    slab_cart_proj = np.sqrt( (track_x[i] - track_x[i - 1])**2 + (track_y[i] - track_y[i - 1])**2 )
                    slab_length = np.sqrt( (track_x[i] - track_x[i - 1])**2 + (track_y[i] - track_y[i - 1])**2 + (track_z[i] - track_z[i - 1])**2)
                    
                elif coordinate_system == 'Spherical':
                    fwd_az, back_az, slab_cart_proj_m = geodesic.inv(track_x[i-1], track_y[i-1], track_x[i], track_y[i])
                    slab_cart_proj = slab_cart_proj_m / 1e3
                    slab_length = np.sqrt( slab_cart_proj**2 + (track_z[i] - track_z[i - 1])**2)
                    
                riserun = (track_z[i - 1] - track_z[i]) / (slab_cart_proj)
                segment_length.append(slab_length * 1000)
                dip_holder.append(np.rad2deg(np.arctan(abs(riserun))))
                
        dips = []
        for k in range(len(dip_holder) - 1):
            if k != int(len(dip_holder) - 1):
                dips.append([dip_holder[k], dip_holder[k + 1]])
            else:
                dips.append([dip_holder[k]])
        
        # Write to the World Builder File
        world_builder_file.write('    {"coordinate":' + str(index - 1) + ',\n')
        world_builder_file.write('     "segments":[')
        for current_segment in range(segment_num):
            world_builder_file.write('    ' + segment_string(segment_length, segment_thickness, dips, top_truncation, index - 1, total_sections, segment_num, current_segment))